In [ ]:
# Importieren der benötigten Pakete
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import numpy as np
import pickle
import time






# Öffnen der Input Datei um sie in Binär zu lesen (rb = read-binary)
pickle_in = open("X.pickle","rb")
# Geöffnete Datei in das Program laden
X = pickle.load(pickle_in)

# Selbe nochmal nur mit der Output Layer
pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

# Den Output in einen Numpy Array stecken um es später zu verwenden
y = np.array(y)
# Input(Bild Pixel) durch 255(rgb) dividieren um Graues Bild zu bekommen
X = X/255.0


dense_layer = [1]
layer_size = [64]
conv_layer = [3]



for dense_layer in dense_layer:
    for layer_size in layer_size:
        for conv_layer in conv_layer:
            # Definieren des Datei-Namen
            NAME = "{}-conv-{}-nodes-{}-dense".format(conv_layer, layer_size, dense_layer)
            # Ein Tensorboard erstellen
            tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs/{}'.format(NAME))

            print(NAME)

            # Sequential wird für Layers benutzt die nur einen Input und einen Output haben
            model = Sequential()

            # Hinzufügen einer "Hidden Layer" mit der obigen definierten Größe 64, (3,3) Kerne und dem im Script vorher definierten Shape
            model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]))
            # Activation Layer ist relu sprich es gibt uns eine lineare Funktion aus wenn der Input Positiv ist
            model.add(Activation('relu'))
            # Das Maximum-Pooling oder Max-Pooling ist eine Pooling-Operation, bei der der maximale oder größte Wert berechnet wird
            model.add(MaxPooling2D(pool_size=(2, 2)))
            
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))
            
            # Alle Dimensionen bis auf eine entfernen
            model.add(Flatten()) 
            
            for l in range(dense_layer):
                model.add(Dense(512))
                model.add(Activation('relu'))


            # Output Layer
            model.add(Dense(1))
            # Die Sigmoid Kurve ähnelt der Form eines S und kann einen Wert zwischen 0 und 1 annehmen (loss and accuracy)
            model.add(Activation('sigmoid'))
    
    
            # Binary_Crossentropy gibt einen Output von 0 oder 1
            # Der Adamoptimizer ist ein optimizer in TensorFlow, der eine adaptive Lernrate verwendet, um die Gewichte in einem neuronale Netzwerk zu aktualisieren
            # Es soll nach genauigkeit trainieren und diese aufzeichnen
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])
            # Anfangen das Model mit den Pickle dateien für 10 Epochen zu belehren. Außerdem die Logs auf das Tensorboard speicher und anzeigen
            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=tensorboard)
            
            model.save('64x3-CNN.model')
       